<a href="https://colab.research.google.com/github/misrori/goldhand_apps/blob/main/binance_strategy_tester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/misrori/binance_strategy

Cloning into 'binance_strategy'...
remote: Enumerating objects: 1172, done.
remote: Counting objects: 100% (305/305), done.
remote: Compressing objects: 100% (303/303), done.
remote: Total 1172 (delta 2), reused 305 (delta 2), pack-reused 867
Receiving objects: 100% (1172/1172), 713.54 MiB | 10.49 MiB/s, done.
Resolving deltas: 100% (47/47), done.
Updating files: 100% (825/825), done.


In [2]:
!pip install goldhand

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.7 MB/s eta 0:00:00
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=fd22ff8eeeffd63ba0c89280f98b1fb1dc2e0933d10442dc897e3468bc909f86
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [3]:
import pandas as pd
import pandas_ta as ta
import os
import warnings
warnings.filterwarnings('ignore')

def smma(data, window, colname):
    hl2 = data['hl2'].values
    smma_values = [hl2[0]]

    for i in range(1, len(hl2)):
        smma_val = (smma_values[-1] * (window - 1) + hl2[i]) / window
        smma_values.append(smma_val)

    data[colname] = smma_values
    return data


def read_one_file(fpath):

    #t = "/content/binance_startegy/data/BTCUSDT_1h.csv"

    df= pd.read_csv(fpath)
    df['date'] = pd.to_datetime(df['date'])
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['close'] = df['close'].astype(float)

    # Rsi
    df['rsi'] = ta.rsi(df['close'], 14)

    # SMAS
    df['sma_50']= ta.sma(df['close'], 50)
    df['diff_sma50'] = (df['close']/df['sma_50'] -1)*100
    df['sma_100']= ta.sma(df['close'], 100)
    df['diff_sma100'] = (df['close']/df['sma_100'] -1)*100
    df['sma_200']= ta.sma(df['close'], 200)
    df['diff_sma200'] = (df['close']/df['sma_200'] -1)*100


    # EMAS
    df['ema_7']= ta.ema(df['close'], 7)
    df['diff_ema7'] = (df['close']/df['ema_7'] -1)*100
    df['ema_14']= ta.ema(df['close'], 14)
    df['diff_ema14'] = (df['close']/df['ema_14'] -1)*100
    df['ema_21']= ta.ema(df['close'], 21)
    df['diff_ema21'] = (df['close']/df['ema_21'] -1)*100

    #Bolinger bands
    bb = ta.bbands(df['close'])
    bb.columns = ['bb_lower', 'bb_mid', 'bb_upper', 'bandwidth', 'percent']
    df['bb_lower'] = bb['bb_lower']
    df['bb_upper'] = bb['bb_upper']
    df['diff_upper_bb'] = (df['bb_upper']/df['close'] -1)*100
    df['diff_lower_bb'] = (df['bb_lower']/df['close'] -1)*100


    # Apply SMMA to the dataframe
    df['hl2'] = (df['high'] + df['low'])/2
    df = smma(df, 7, 'v1')
    df = smma(df, 11, 'v2')
    df = smma(df, 16, 'v3')
    df = smma(df, 21, 'v4')
    return df


In [4]:
all_file = os.listdir('/content/binance_strategy/data/')

hourly_data = [f"/content/binance_strategy/data/{ticker}" for ticker in all_file if ticker.endswith('1h.csv')]
quater_hour_data = [f"/content/binance_strategy/data/{ticker}" for ticker in all_file if ticker.endswith('15m.csv')]



In [ ]:
# your strategy


In [5]:
from goldhand import *
from tqdm import tqdm

all_summary = []
all_trades = []

for data_file in tqdm(hourly_data):
    try:
        ticker = data_file.split('/')[4].split('_')[0]

        data = read_one_file(data_file)

        backtest = Backtest( data, rsi_strategy, buy_threshold=30, sell_threshold=70)

        t_summary =  backtest.trades_summary
        t_trades =  backtest.trades

        t_summary['ticker'] = ticker
        t_trades['ticker'] = ticker

        all_summary.append(t_summary)
        all_trades.append(t_trades)
    except:
       pass

summary_df = pd.DataFrame(all_summary)
tradesdf = pd.concat(all_trades, ignore_index=True)


100%|██████████| 411/411 [02:29<00:00,  2.75it/s]


In [ ]:

tradesdf['result'].mean()

1.0047996232203578

In [ ]:
summary_df.sort_values(by=['median_res(%)', 'number_of_trades'], ascending=[False, False])

,ticker,number_of_trades,win_ratio(%),average_res(%),median_res(%),cumulative_result,trade_results,profitable_trade_results,profitable_trades_mean,profitable_trades_median,...,looser_trades_median,average_trade_len(days),median_trade_len(days),number_of_win_trades,number_of_lost_trades,max_gain(%),max_lost(%),first_trade_buy,buy_threshold,sell_threshold
18,AEURUSDT,2,100.00,57.59,57.59,2.167302,113.82 # 1.36,113.82 # 1.36,57.59,57.59,...,NaN,1.500000,1.5,2,0,113.82,1.36,2023-12-05 01:00:00,30,70
341,ORDIUSDT,4,100.00,11.82,13.33,1.558445,14.87 # 4.58 # 16.04 # 11.79,14.87 # 4.58 # 16.04 # 11.79,11.82,13.33,...,NaN,2.500000,2.5,4,0,16.04,4.58,2023-11-21 02:00:00,30,70
91,1000SATSUSDT,1,100.00,11.62,11.62,1.116214,11.62,11.62,11.62,11.62,...,NaN,7.000000,7.0,1,0,11.62,11.62,2023-12-18 10:00:00,30,70
3,JTOUSDT,3,100.00,7.42,9.12,1.237823,10.23 # 9.12 # 2.91,10.23 # 9.12 # 2.91,7.42,9.12,...,NaN,1.000000,1.0,3,0,10.23,2.91,2023-12-18 07:00:00,30,70
20,BEAMXUSDT,4,75.00,9.41,8.70,1.412872,11.16 # 22.89 # 6.24 # -2.65,11.16 # 22.89 # 6.24,13.43,11.16,...,-2.65,3.750000,3.5,3,1,22.89,-2.65,2023-11-21 22:00:00,30,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,GLMRUSDT,52,40.38,-4.68,-1.67,0.038318,45.43 # 1.16 # -26.55 # -11.7 # -44.6 # -6.38 ...,45.43 # 1.16 # 15.98 # 3.9 # 5.67 # 7.24 # 11....,7.40,4.44,...,-6.38,7.423077,6.0,21,31,45.43,-46.26,2022-01-12 06:00:00,30,70
115,ARKUSDT,9,44.44,0.84,-2.32,0.980835,4.8 # -8.24 # 9.18 # -2.32 # 26.15 # 14.14 # -...,4.8 # 9.18 # 26.15 # 14.14,13.57,11.66,...,-6.26,4.888889,5.0,4,5,26.15,-27.19,2023-09-24 00:00:00,30,70
48,MEMEUSDT,5,20.00,-2.61,-2.88,0.850383,-4.54 # -2.88 # 14.15 # -0.71 # -19.07,14.15,14.15,14.15,...,-3.71,5.800000,3.0,1,4,14.15,-19.07,2023-11-05 13:00:00,30,70
100,BONKUSDT,2,0.00,-10.61,-10.61,0.796634,-5.78 # -15.45,,NaN,NaN,...,-10.61,5.000000,5.0,0,2,-5.78,-15.45,2023-12-15 22:00:00,30,70


In [ ]:
summary_df.sort_values(by='cumulative_result', ascending=False)

,number_of_trades,average_res(%),median_res(%),cumulative_result,trade_results,profitable_trade_results,profitable_trades_mean,profitable_trades_median,looser_trade_results,looser_trades_mean,...,median_trade_len(days),win_ratio(%),number_of_win_trades,number_of_lost_trades,max_gain(%),max_lost(%),first_trade_buy,buy_threshold,sell_threshold,ticker
174,71,5.85,4.33,25.080362,114.58 # 23.72 # 21.34 # 8.8 # 19.53 # 5.1 # 1...,114.58 # 23.72 # 21.34 # 8.8 # 19.53 # 5.1 # 1...,10.89,5.84,-1.26 # -28.37 # -8.81 # -4.53 # -34.86 # -9.2...,-10.17,...,3.0,76.06,54,17,114.58,-34.86,2021-07-24 00:00:00,30,70,DEXEUSDT
325,69,4.61,5.47,12.873259,-3.83 # 4.19 # 27.08 # -0.18 # 6.73 # 17.79 # ...,4.19 # 27.08 # 6.73 # 17.79 # 13.23 # 13.76 # ...,9.43,7.93,-3.83 # -0.18 # -6.72 # -1.54 # -18.48 # -0.7 ...,-9.04,...,4.0,73.91,51,18,36.68,-54.51,2021-10-05 17:00:00,30,70,CVPUSDT
279,77,4.06,5.06,11.868129,17.96 # 37.78 # 5.05 # -12.41 # 16.92 # -14.94...,17.96 # 37.78 # 5.05 # 16.92 # 19.42 # 8.23 # ...,9.74,8.13,-12.41 # -14.94 # -15.73 # -12.78 # -2.07 # -5...,-10.12,...,4.0,71.43,55,22,37.78,-45.08,2021-04-16 12:00:00,30,70,OGUSDT
224,88,3.56,4.44,11.150117,1.47 # 8.96 # 10.4 # 2.56 # -25.28 # 9.74 # 2....,1.47 # 8.96 # 10.4 # 2.56 # 9.74 # 2.41 # 11.2...,9.42,9.12,-25.28 # -23.7 # -6.57 # -3.91 # -9.81 # -17.1...,-11.20,...,3.0,71.59,63,25,31.95,-34.19,2021-04-10 16:00:00,30,70,INJUSDT
377,68,4.50,6.56,9.178618,-17.28 # -1.02 # -48.92 # 17.3 # 19.08 # -25.8...,17.3 # 19.08 # 17.15 # 12.06 # 4.61 # 10.83 # ...,10.99,9.60,-17.28 # -1.02 # -48.92 # -25.81 # -23.38 # -1...,-14.99,...,2.5,75.00,51,17,37.20,-48.92,2021-04-07 11:00:00,30,70,COCOSUSDT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,81,-2.50,1.67,0.061020,0.14 # 7.78 # -6.49 # 6.64 # -45.09 # 16.9 # -...,0.14 # 7.78 # 6.64 # 16.9 # 2.36 # 2.18 # 7.33...,5.46,5.07,-6.49 # -45.09 # -6.21 # -11.91 # -16.67 # -16...,-14.08,...,5.0,58.02,47,34,16.90,-45.09,2021-04-06 14:00:00,30,70,FILUSDT
333,82,-2.72,1.19,0.040212,11.79 # 4.09 # 9.81 # 0.66 # -42.19 # 5.71 # -...,11.79 # 4.09 # 9.81 # 0.66 # 5.71 # 4.16 # 9.6...,6.53,5.76,-42.19 # -16.17 # -12.07 # -20.67 # -15.17 # -...,-14.54,...,4.5,56.10,46,36,23.25,-47.15,2021-04-07 10:00:00,30,70,RSRUSDT
222,52,-4.68,-1.67,0.038318,45.43 # 1.16 # -26.55 # -11.7 # -44.6 # -6.38 ...,45.43 # 1.16 # 15.98 # 3.9 # 5.67 # 7.24 # 11....,7.40,4.44,-26.55 # -11.7 # -44.6 # -6.38 # -1.62 # -28.8...,-12.86,...,6.0,40.38,21,31,45.43,-46.26,2022-01-12 06:00:00,30,70,GLMRUSDT
244,63,-1.36,1.97,0.038192,-4.09 # -40.09 # 6.17 # 1.87 # -24.96 # 5.47 #...,6.17 # 1.87 # 5.47 # 13.04 # 9.97 # 9.26 # 4.4...,6.80,5.55,-4.09 # -40.09 # -24.96 # -8.93 # -19.07 # -16...,-12.25,...,4.0,57.14,36,27,25.51,-94.89,2021-04-16 09:00:00,30,70,FTTUSDT


In [ ]:
from IPython.display import Javascript
from goldhand import *
from tqdm import tqdm
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 8000})'''))

symbol = 'COCOSUSDT'

data = read_one_file(f'/content/binance_strategy/data/{symbol}_1h.csv')

backtest = Backtest( data, rsi_strategy, buy_threshold=30, sell_threshold=70)

t_summary =  backtest.trades_summary
t_trades =  backtest.trades
backtest.summarize_strategy()


<IPython.core.display.Javascript object>

,Strategy summary
number_of_trades,68
average_res(%),4.5
median_res(%),6.56
cumulative_result,9.178618
trade_results,-17.28 # -1.02 # -48.92 # 17.3 # 19.08 # -25.8...
profitable_trade_results,17.3 # 19.08 # 17.15 # 12.06 # 4.61 # 10.83 # ...
profitable_trades_mean,10.99
profitable_trades_median,9.6
looser_trade_results,-17.28 # -1.02 # -48.92 # -25.81 # -23.38 # -1...
looser_trades_mean,-14.99


,result,buy_price,sell_price,buy_date,sell_date,days_in_trade,trade_id,status,buy_open,buy_high,...,sell_diff_ema21,sell_bb_lower,sell_bb_upper,sell_diff_upper_bb,sell_diff_lower_bb,sell_hl2,sell_v1,sell_v2,sell_v3,sell_v4
0,0.827234,1.2925,1.0692,2021-04-07 11:00:00,2021-04-27 21:00:00,20,1,closed,1.3884,1.3897,...,10.861913,0.922122,1.074038,0.452501,-13.755905,1.04245,0.972382,0.959445,0.944680,0.931159
1,0.989760,1.1328,1.1212,2021-05-04 15:00:00,2021-05-11 22:00:00,7,2,closed,1.1724,1.1731,...,12.956768,0.912956,1.121604,0.036032,-18.573313,1.06465,0.991066,0.986748,0.991357,0.999236
2,0.510801,0.8518,0.4351,2021-05-12 23:00:00,2021-05-24 20:00:00,11,3,closed,0.9406,0.9482,...,13.829248,0.372495,0.438705,0.828639,-14.388740,0.42440,0.389975,0.381145,0.378728,0.382568
3,1.172984,0.4538,0.5323,2021-05-28 11:00:00,2021-05-30 11:00:00,2,4,closed,0.4798,0.4827,...,15.386554,0.436999,0.533841,0.289539,-17.903667,0.53210,0.465031,0.459509,0.461373,0.466319
4,1.190776,0.4445,0.5293,2021-06-08 02:00:00,2021-06-10 19:00:00,2,5,closed,0.4684,0.4684,...,15.184505,0.400000,0.529560,0.049130,-24.428499,0.49660,0.460993,0.457555,0.455008,0.454109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,1.057420,1.6301,1.7237,2023-04-21 04:00:00,2023-04-23 03:00:00,1,64,closed,1.6547,1.6710,...,5.514355,1.657407,1.736353,0.734061,-3.845971,1.72500,1.650361,1.631178,1.626758,1.630501
64,1.156909,1.6602,1.9207,2023-04-25 15:00:00,2023-04-26 09:00:00,0,65,closed,1.6823,1.6823,...,10.018435,1.706778,1.927342,0.345835,-11.137733,1.88185,1.756265,1.741515,1.738832,1.738905
65,1.095975,1.6598,1.8191,2023-05-03 15:00:00,2023-05-04 04:00:00,0,66,closed,1.6676,1.6686,...,4.033722,1.755866,1.838614,1.072713,-3.476100,1.82850,1.760564,1.745764,1.739713,1.738886
66,0.845682,1.6291,1.3777,2023-05-06 14:00:00,2023-05-16 10:00:00,9,67,closed,1.6692,1.6699,...,4.945580,1.277674,1.394366,1.209731,-7.260395,1.36500,1.318300,1.310823,1.304393,1.299408
